In [2]:
from dask.distributed import Client, LocalCluster
from dask import delayed, dataframe as dd
import dask

import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import math
import numpy as np

import momepy
from scripts.tessellation import enclosed_tessellation

import matplotlib.pyplot as plt
import geopandas as gpd

/tmp/ipykernel_1889/1812556411.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
place = "singapore"
local_crs = 3414

In [4]:
Daskcluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(Daskcluster)
client

2023-05-03 18:37:55,125 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,129 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,132 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,136 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,138 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,141 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,144 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-03 18:37:55,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39003,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:35727,Total threads: 2
Dashboard: http://127.0.0.1:38409/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:33385,


In [5]:
streets = gpd.read_parquet(f"./out/{place}/streets.pq").explode()

buildings = gpd.read_parquet(f"./out/{place}/buildings_raw.pq")

study_area = gpd.read_parquet(f"./out/{place}/study_area.pq")

water = gpd.read_parquet(f"./out/{place}/water.pq")

/tmp/ipykernel_1889/2964440053.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet(f"./out/{place}/streets.pq").explode()


In [9]:
water_bodies_boundaries = []

# Explode the GeoDataFrame into a GeoSeries of polygons and multipolygons
gs = water.explode()

# Convert each polygon into a MultiLineString
mls = gs.geometry.boundary

# Convert the MultiLineString into a DataFrame
water_bodies = gpd.GeoDataFrame({'geometry': mls})

for water_body in water_bodies.geometry:
    # Create a LineString object from the coordinates
    water_bodies_boundaries.append(water_body)

# # Create a GeoDataFrame of the water area boundaries
water_bodies_boundaries = gpd.GeoDataFrame({'geometry': water_bodies_boundaries})

/tmp/ipykernel_1889/2503369575.py:4: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gs = water.explode()


In [10]:
water_bodies_boundaries

,geometry
0,"LINESTRING (37698.921 36379.463, 37801.240 364..."
1,"LINESTRING (10638.104 45329.526, 10576.655 452..."
2,"LINESTRING (16251.515 32735.477, 16202.884 325..."
3,"LINESTRING (20392.301 40523.349, 20411.654 405..."
4,"LINESTRING (19219.340 37721.260, 19188.801 376..."
...,...
1449,"LINESTRING (32275.071 39839.324, 32386.436 398..."
1450,"LINESTRING (32275.071 39839.324, 32275.260 398..."
1451,"MULTILINESTRING ((15995.367 34754.843, 15992.4..."
1452,"LINESTRING (43523.381 43174.406, 43529.379 431..."


In [ ]:
# study_area = gpd.GeoDataFrame(pd.concat( [study_area, streets_buffer], ignore_index=True)).dissolve()

In [ ]:
# # Create a figure and axis
# fig, ax = plt.subplots(figsize=(100, 100))

# # Plot study_area in green on the same axis
# study_area.plot(ax=ax, color='green')

# # Plot streets in blue on the same axis
# streets.plot(ax=ax, color='blue')

# # Plot buildings in red on the same axis
# buildings.plot(ax=ax, color='red')

# outlines.plot(ax=ax, color='black')

# # Show the plot
# plt.show()

In [ ]:
enclosures = momepy.enclosures(streets.append(water_bodies_boundaries), limit= study_area, clip=True)

In [ ]:
tessellation = momepy.Tessellation(buildings, unique_id='uID', enclosures=enclosures, use_dask = True).tessellation

# for debugging
tessellation_copy = tessellation

In [ ]:
tessellation = tessellation_copy

In [ ]:
tessellation.to_parquet(f"./out/{place}/tessellation_raw.pq")

In [35]:
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_raw.pq")

In [36]:
tessellation = tessellation.drop("eID", axis = 1)
tessellation.dropna()

,uID,geometry
0,75707.0,"POLYGON ((45510.911 33224.180, 45517.076 33236..."
1,74906.0,"MULTIPOLYGON (((43584.161 35868.253, 43584.343..."
2,74909.0,"MULTIPOLYGON (((43416.661 36165.753, 43415.986..."
3,74915.0,"MULTIPOLYGON (((43506.033 36382.388, 43505.789..."
4,74921.0,"POLYGON ((43453.691 36562.605, 43453.334 36563..."
...,...,...
87401,35292.0,"POLYGON ((27588.025 38604.710, 27587.246 38604..."
87414,31010.0,"POLYGON ((26730.051 26703.640, 26729.051 26704..."
87434,7729.0,"POLYGON ((16049.921 34749.500, 16050.812 34752..."
87461,36329.0,"POLYGON ((27847.387 22035.537, 27847.387 22125..."


In [37]:
tessellation = tessellation.overlay(water, how="difference")

In [38]:
tessellation = tessellation.dropna()

In [39]:
tessellation = tessellation.explode().reset_index().drop(["level_0", "level_1"], axis=1)

/tmp/ipykernel_21090/109688082.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  tessellation = tessellation.explode().reset_index().drop(["level_0", "level_1"], axis=1)


In [40]:
@dask.delayed
def find_largest(group, building):
    if len(group) == 1:
        return gpd.GeoDataFrame(group)
    else:
        intersects = gpd.sjoin(group, building.set_crs(local_crs), predicate='intersects')
        if len(intersects) == 1:
            return gpd.GeoDataFrame(intersects)
        else:
            return gpd.GeoDataFrame(intersects.loc[[intersects.area.idxmax()]])
        

In [41]:
delayed_obj = []

for uID, group in tessellation.groupby("uID"):
    building = buildings.loc[buildings["uID"] == uID]
    delayed_obj.append(find_largest(group, building))
    
objects = dask.compute(*delayed_obj)

/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 483.56 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/virgilxw/miniconda3/lib/python3.10/site-packages/cloudpickle/cloudpickle.py:649: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/m

In [42]:
tessellation = gpd.GeoDataFrame(pd.concat(list(objects)), crs=tessellation.crs).reset_index(drop=True)

In [43]:
tessellation = tessellation.join(buildings, on='uID', how='inner', rsuffix="right")

In [44]:
tessellation["new_uID"] = range(len(tessellation))

In [45]:
buildings = pd.merge(buildings, tessellation, how='inner', left_on='uID', right_on='uID')

In [46]:
buildings["uID"] = buildings["new_uID"]
tessellation["uID"] = tessellation["new_uID"]

In [47]:
buildings.drop(["geometry_y", "geometryright", "uIDright", "new_uID"], axis = 1, inplace = True)
buildings.rename(columns = {"geometry_x": "geometry"}, inplace = True)

In [48]:
buildings = gpd.GeoDataFrame(buildings).set_crs(tessellation.crs, allow_override=True)

In [49]:
buildings = buildings.sort_values(by='uID', ascending=True)

In [1]:
tessellation = tessellation.sort_values(by='uID', ascending=True)
tessellation.drop(["geometryright", "uIDright", "new_uID"], axis = 1, inplace = True)
tessellation = tessellation.to_crs(local_crs)
tessellation = tessellation.reset_index(drop=True)

NameError: name 'tessellation' is not defined

In [28]:
Daskcluster.close()
client.shutdown()

In [56]:
buildings.iloc[buildings["uID"].idxmax()]

geometry       POLYGON ((50664.41540419368 44793.93508395505,...
uID                                                        75038
uID_left                                                     NaN
index_right                                                  NaN
uID_right                                                    NaN
Name: 75038, dtype: object

In [29]:
tessellation.to_parquet(f"./out/{place}/tessellation.pq")

In [30]:
buildings.to_parquet(f"./out/{place}/buildings.pq")